<h1> Name: Eshan Mehrotra

## <font color=blue>*Exercise 1 Time as Strings*</font>
### Parsing Strings/Tupes to get Hours and/or Mins and/or Seconds

In [1]:
def parse_hms(time):
    '''
    Convert String input of a time duration and return tuple of hours, mins & seconds.
    Can parse inputs without hours or mins time information.
    Example input-outputs:
    '23' returns (0,0,23)
    '45:00' returns (0,45,0)
    '2:34:16' returns (2,34,16)
    '''
    if (time.count(':')==0):
        fulltime = time.rjust(4,':')
    elif (time.count(':')==1):
        fulltime = time.rjust(6,':')
    else:
        fulltime = time
    
    h,m,s = ['00' if x.strip()=='' else x for x in fulltime.split(':')]
    return (int(h),int(m),int(s))

In [2]:
print(parse_hms('2:34:16'))
print(parse_hms('14:21:56'))
print(parse_hms('45:00'))
print(parse_hms('23'))
print(parse_hms('12::13'))

(2, 34, 16)
(14, 21, 56)
(0, 45, 0)
(0, 0, 23)
(12, 0, 13)


In [3]:
def hms_to_seconds(h,m,s):
    '''
    Converts hours, mins and seconds into its equivalent number of seconds and returns it.
    Example input-outputs:
    (1,23,24) returns 5004
    '''
    return ((h*3600) + (m*60) + s)

In [4]:
print(hms_to_seconds(1,23,24))
print(hms_to_seconds(0,33,37))

5004
2017


In [5]:
def seconds_to_hms(sec):
    '''
    Converts total number of seconds into equivalent tuple of (hours,mins,seconds) and returns it. 
    Example input-outputs:
    (2017) returns (0,33,37)
    '''
    return (((sec//60)//60,(sec//60)%60, sec%60))

In [6]:
print(seconds_to_hms(2017))
print(seconds_to_hms(0))
print(hms_to_seconds(*seconds_to_hms(54321)))

(0, 33, 37)
(0, 0, 0)
54321


In [7]:
def sum_hms(timearr):
    '''
    Take the array of time duration strings & adds them returning a tuple of (h,m,s).
    Example input-output:
    ['0:1:24', '2:34:45'] returns (2, 36, 9)
    [] returns (0,0,0)
    '''
    allseconds = [hms_to_seconds(*parse_hms(time)) for time in timearr]
    
    return (seconds_to_hms(sum(allseconds)))
            

In [8]:
print(sum_hms(['0:1:24', '2:34:45']))
print(sum_hms(['3:00:00', '23:00', '45']))
print(sum_hms([]))

(2, 36, 9)
(3, 23, 45)
(0, 0, 0)


<a id='varincomp'></a>
## <font color=blue>*Exercise 2 Amicable Numbers*</font>
### Find all Amicable Numbers less than or equal to n. 
**Amicable Numbers**(https://en.wikipedia.org/wiki/Amicable_numbers)

In [9]:
def proper_divisors(n):
    '''
    Returns a list of all proper divisors of n.
    proper divisor of a number is a positive factor of that number other than the number itself. 
    For example, the proper divisors of 6 are 1, 2, and 3
    '''
    return [i for i in range(1,n) if n%i == 0]

In [10]:
def amicable_numbers(n):
    '''
    Returns pairs of amicable numbers less than or equal to n.
    2 numbers are amicable if the sum of the proper divisos of each is equal to the other number.
    Example: (220, 284). 
    They are amicable because the proper divisors of 220 are 1, 2, 4, 5, 10, 11, 20, 22, 44, 55 and 110, 
    of which the sum is 284;
    and the proper divisors of 284 are 1, 2, 4, 71 and 142, of which the sum is 220
    '''
    amic = [(i,j) for i in range(1,n) 
           for j in (sum(proper_divisors(i)),) 
           if (j>i) & (sum(proper_divisors(j)) == i)]
    return amic

In [11]:
print(amicable_numbers(1000))
print(amicable_numbers(10000))

[(220, 284)]
[(220, 284), (1184, 1210), (2620, 2924), (5020, 5564), (6232, 6368)]


In [12]:
%timeit -r1 amicable_numbers(1000)
%timeit -r1 amicable_numbers(10000)

37.9 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)
4.14 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<a id='dictjoin'></a>
## <font color=blue>*Exercise 3 File I/O & Dicts*</font>
### Read MovieLens data, merge files, calc avg movie rating & write to file  

In [13]:
from collections import defaultdict
from statistics import mean
import os

In [14]:
def read_u_data(data_file='u.data', encoding='utf-8'):
    '''
    Read the data from the file and return the details as a Dictionary containing a Movie with MovieID as key
    and a list of all of it's corresponding ratings as the values.
    '''
    with open(data_file,'r',encoding=encoding) as data:
        datalst = [line.split() for line in data]
     
    ratings = defaultdict(list)
    for _,key,val,*_ in datalst:
        ratings[key].append(int(val))    
        
    return ratings

In [15]:
def calculate_avg_u_data_rating(d):
    '''
    Takes a dictionary & returns another dictionary with Movie and the corresponding average of all of it's ratings.
    '''
    return {key:round(mean(d.get(key)),3) for key in d.keys()}

In [16]:
def read_u_item(data_file='u.item', encoding='latin-1'):
    '''
    Read the data from the file and return the details as a Dictionary containing a Movie with MovieID as key
    and its title as the values.
    '''
    with open(data_file,'r',encoding=encoding) as item:
        itemlst = [line.split("|") for line in item]
        
    return {itemid:title for itemid,title,*_ in itemlst}

In [17]:
def write_ratings(u_data_avg, u_item, fname='movie-ratings.txt'):
    '''
    Joins the two dictionaries given and writes output a txt file with the movieid,avg-rating & movie title
    details.
    '''
    
    allinfo = [(k,round(u_data_avg[k],2),u_item[k]) for k in u_data_avg.keys()]
    allinfo_sorted = sorted(allinfo, key = lambda x: (-x[1],x[2]))
    
    with open(fname, "w") as output:
        
        output.write('\n'.join('   '.join(map(str, movie)) for movie in allinfo_sorted))
    
    return allinfo_sorted

In [18]:
d1 = read_u_data()
d2 = calculate_avg_u_data_rating(d1)
u = read_u_item()
write_ratings(d2, u)[:10]

[('1536', 5, 'Aiqing wansui (1994)'),
 ('1653', 5, 'Entertaining Angels: The Dorothy Day Story (1996)'),
 ('814', 5, 'Great Day in Harlem, A (1994)'),
 ('1201', 5, 'Marlene Dietrich: Shadow and Light (1996) '),
 ('1189', 5, 'Prefontaine (1997)'),
 ('1467', 5, 'Saint of Fort Washington, The (1993)'),
 ('1500', 5, 'Santa with Muscles (1996)'),
 ('1599', 5, "Someone Else's America (1995)"),
 ('1293', 5, 'Star Kid (1997)'),
 ('1122', 5, 'They Made Me a Criminal (1939)')]

### Total of 1682 lines in the output file as that is the count of the unique number of movies in the dataset. 

## <font color=blue>*Exercise 4 One Hot Encoding*</font>
### Encoding N nominal values using N bits 

In [19]:
def one_hot_encoding(nomarr):
    '''
    Takes the N unique nominal values in the nomarray and encodes each value with N bits.
    1 bit is set for each unique value.
    Example: 
    ['alpha','beta', 'gamma'] 
    returns 
    [['alpha', [1, 0, 0]], ['beta', [0, 1, 0]], ['gamma', [0, 0, 1]]]
    '''
    
    results= []
    for item in nomarr:
        onehotitem = [0]*len(set(nomarr))
        onehotitem[nomarr.index(item)] = 1
        results.append([item,onehotitem])
    return results
    

In [20]:
one_hot_encoding(['ford','honda', 'toyota', 'mazda', 'subaru'])

[['ford', [1, 0, 0, 0, 0]],
 ['honda', [0, 1, 0, 0, 0]],
 ['toyota', [0, 0, 1, 0, 0]],
 ['mazda', [0, 0, 0, 1, 0]],
 ['subaru', [0, 0, 0, 0, 1]]]

In [21]:
one_hot_encoding(['alpha','beta', 'gamma','alpha'])

[['alpha', [1, 0, 0]],
 ['beta', [0, 1, 0]],
 ['gamma', [0, 0, 1]],
 ['alpha', [1, 0, 0]]]

<a id='enumerate'></a>
## <font color=blue>*Exercise 5 Dense to Sparse Matrix Conversion*</font>
### Represent dense matrix as a list of triplet of form (row index,column index, value) 

In [22]:
def dense2sparse(m):
    '''
    Takes a dense matrix and convert the matrix into a sparse form where the non-zero values are stored as a list of 
    triplet of form (row index, column index, value). The order of the triplets does not matter
    '''
    ans = [(i,j,val) for i,row in enumerate(m) 
                     for j,val in enumerate(row) if val != 0]
    return ans

In [23]:
print(dense2sparse([[1, 0, 2],[0, 0, 3]]))
print(dense2sparse([[1, 0, 2],[0, 0, 3],[0,1,2]]))

[(0, 0, 1), (0, 2, 2), (1, 2, 3)]
[(0, 0, 1), (0, 2, 2), (1, 2, 3), (2, 1, 1), (2, 2, 2)]


## <font color=blue>*Exercise 6 Probability of Dice Throws*</font>
### Simulate throws of dices and compute probability of required events 

In [24]:
import random

In [25]:
def dice():
    '''
    Simulate the throw of dice by returning a random number between 1 and 6
    '''
    return random.randint(1,6)
    

In [26]:
def six_in_4():
    '''
    Simulate 4 dice throws and returns True if 6 occurs, False if it doesn't
    '''
    occur = 6 in [dice() for i in range(1,5)]
    return occur

In [27]:
def two_six_in_24():
    '''
    Simulate 24 throws of 2 dices and returns True if pair of 6's occur (6,6), otherwise false
    '''
    occur = (6,6) in [(dice(),dice()) for i in range(1,25)]
    return occur

In [28]:
def prob_question(n=1000):
    '''
    Perform the 2 simulations n times and return the prob of occurance of both events as a tuple 
    of form (prob of event 1, prob of event 2).
    If no n specified, it default to 1000 simulations.
    '''
    event1 = [six_in_4() for i in range(1,1+n)].count(True)
    event2 = [two_six_in_24() for i in range(1,1+n)].count(True)
    return ((event1/n),(event2/n))

In [29]:
print(prob_question(n=10000))

(0.5154, 0.4959)


<a id='closure'></a>
## <font color=blue>*Exercise 7 Closure Algorithm*</font>
### Find the Closure of a Set of Attributes

In [30]:
def lhs(tup):
    '''
    Split the tuple FD and return the Left Hand Side of the FD as set
    '''
    return tup[0]

In [31]:
def rhs(tup):
    '''
    Split the tuple FD and return the Right Hand Side of the FD as set
    '''
    return tup[1]

In [32]:
def parse_fd(fd):
    '''
    Takes the String representation of a functional dependency and returns a tuple of sets
    representing LHS and RHS of the FD. 
    '''

    tupfd = (set(fd.split('->')[0].strip().split()),set(fd.split('->')[1].strip().split()))
    return tupfd

In [33]:
def closure(FD, att):
    '''
    Takes the set of attributes att and all Functional Dependencies FD to determine the closure of the given set. 
    It returns the closure set of the given attributes.  
    '''
 
    attset = set(att.strip().split())
    attlst = [a.strip() for a in att.strip().split()]
    newlst = []
    fds = [parse_fd(line.strip()) for line in FD.split('\n')]


    for fd in fds:
        if lhs(fd).issubset(attset):
            newlst.append(next(iter(rhs(fd))))
            newlst += [next(iter(rhs(fd))) for fd in fds 
                                            if lhs(fd).issubset(set(newlst))]  
 
    return (set(newlst+attlst))
        

In [34]:
FDs_1 = ''' A B -> C 
A -> D 
D -> E 
A C -> B '''

closure(FDs_1, 'A')

{'A', 'D', 'E'}

In [35]:
closure(FDs_1, 'A B')

{'A', 'B', 'C', 'D', 'E'}

In [36]:
closure(FDs_1, 'B')

{'B'}

## References
1. **[Ex2 ln[10]](#varincomp)** https://stackoverflow.com/questions/26672532/how-to-set-local-variable-in-list-comprehension/55881984
2. **[Ex3 ln[17]](#dictjoin)** https://stackoverflow.com/questions/5946236/how-to-merge-multiple-dicts-with-same-key-or-different-key
3. **[Ex5 ln[22]](#enumerate)** https://realpython.com/python-enumerate/ 
4. **[Ex7](#closure)** https://vertabelo.com/blog/closure-of-a-set-of-attributes/